In [35]:
import pandas as pd
import numpy as np 

from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [39]:
train = pd.read_csv('nifty_main.csv')
test= pd.read_csv('nifty_test.csv')
train.set_index('Date', inplace=True)
test.set_index('Date', inplace=True)
train.head()
test.head()


,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
22-Jul-10,5441.95,5399.15,5447.15,5372.20,156.00M,0.79%
23-Jul-10,5449.10,5441.90,5477.50,5435.15,165.93M,0.13%
26-Jul-10,5418.60,5446.55,5466.25,5409.20,147.33M,-0.56%
27-Jul-10,5430.60,5420.20,5450.95,5407.20,155.31M,0.22%
28-Jul-10,5397.55,5436.10,5447.85,5386.25,146.94M,-0.61%


,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
23-Aug-18,11582.75,11620.70,11620.70,11546.70,256.16M,0.10%
24-Aug-18,11557.10,11566.60,11604.60,11532.00,225.60M,-0.22%
27-Aug-18,11691.95,11605.85,11700.95,11595.60,205.69M,1.17%
28-Aug-18,11738.50,11731.95,11760.20,11710.50,249.42M,0.40%
29-Aug-18,11691.90,11744.95,11753.20,11678.85,259.35M,-0.40%


In [40]:
train_Price = train.iloc[:, 0:1].values
train_Open = train.iloc[:, 1:2].values
train_High = train.iloc[:, 2:3].values
train_Low =  train.iloc[:, 3:4].values

test_Price = test.iloc[:, 0:1].values
test_Open = test.iloc[:, 1:2].values
test_High = test.iloc[:, 2:3].values
test_Low =  test.iloc[:, 3:4].values

In [41]:
def window_transform_series(series, window_size):
    sc = MinMaxScaler()
    series = sc.fit_transform(series)
    
    x=[]
    y=[]
    
    for i in range(window_size, len(series)):
        x.append(series[i-window_size:i])
        y.append(series[i])
    
    x = np.asarray(x)
    x.shape= (np.shape(x)[0:2])
    y = np.asarray(y)
    y.shape=(len(y), 1)
    
    return x,y                            

In [56]:
windows_size=1
x,y = window_transform_series(train_Open, windows_size)

In [11]:
train_test_split = int(np.ceil(2*len(y)/float(3)))
train_test_split

1334

In [12]:
len(y)

2001

In [13]:
len(train_Open)

2008

In [14]:
len(x)

2001

In [16]:
# this is fpr old profgram 
xtr = x[:train_test_split, :]
ytr = y[:train_test_split]

xts = x[train_test_split:, :]
yts = y[train_test_split:]

xtr = np.reshape(xtr, (xtr.shape[0], windows_size, 1))
xts = np.reshape(xts, (xts.shape[0], windows_size, 1))

In [57]:
# this is for our program 
xtr, ytr = window_transform_series(train_Open, windows_size)
xts, yts = window_transform_series(test_Open, windows_size)

xtr = np.reshape(xtr, (xtr.shape[0], windows_size, 1))
xts = np.reshape(xts, (xts.shape[0], windows_size, 1))

array([[[0.77658487],
        [0.68438991],
        [0.75127812],
        [0.96617246],
        [0.98832652],
        [0.90277778],
        [0.87056919]],

       [[0.68438991],
        [0.75127812],
        [0.96617246],
        [0.98832652],
        [0.90277778],
        [0.87056919],
        [1.        ]]])

In [58]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
import keras

np.random.seed(0)

model = Sequential()
model.add(LSTM(units=5, input_shape = (windows_size, 1)))
model.add(Dense(1))

optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon = 1e-08, decay = 0.0)

model.compile(loss='mean_squared_error', optimizer= optimizer)
model.fit(xtr, ytr, epochs = 500, batch_size=64)


Epoch 1/500
2007/2007 [==============================] - 1s 448us/step - loss: 0.1473
Epoch 2/500
2007/2007 [==============================] - 0s 43us/step - loss: 0.1049
Epoch 3/500
2007/2007 [==============================] - 0s 43us/step - loss: 0.0724
Epoch 4/500
2007/2007 [==============================] - 0s 44us/step - loss: 0.0493
Epoch 5/500
2007/2007 [==============================] - 0s 43us/step - loss: 0.0367
Epoch 6/500
2007/2007 [==============================] - 0s 44us/step - loss: 0.0307
Epoch 7/500
2007/2007 [==============================] - 0s 46us/step - loss: 0.0258
Epoch 8/500
2007/2007 [==============================] - 0s 45us/step - loss: 0.0207
Epoch 9/500
2007/2007 [==============================] - 0s 44us/step - loss: 0.0157
Epoch 10/500
2007/2007 [==============================] - 0s 44us/step - loss: 0.0110
Epoch 11/500
2007/2007 [==============================] - 0s 43us/step - loss: 0.0070
Epoch 12/500
2007/2007 [==============================] - 0s 4

2007/2007 [==============================] - 0s 31us/step - loss: 1.1369e-04
Epoch 93/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1270e-04
Epoch 94/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1536e-04
Epoch 95/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1345e-04
Epoch 96/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1270e-04
Epoch 97/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1284e-04
Epoch 98/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1550e-04
Epoch 99/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1384e-04
Epoch 100/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1328e-04
Epoch 101/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1469e-04
Epoch 102/500
2007/2007 [==============================] - 0s 33us/step - loss: 1.1241e-04
Epoch 103/500
2007/2

2007/2007 [==============================] - 0s 31us/step - loss: 1.1299e-04
Epoch 183/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1173e-04
Epoch 184/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1323e-04
Epoch 185/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1432e-04
Epoch 186/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1207e-04
Epoch 187/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1525e-04
Epoch 188/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1269e-04
Epoch 189/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1098e-04
Epoch 190/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1232e-04
Epoch 191/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1290e-04
Epoch 192/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1363e-04
Epoch 193/500

2007/2007 [==============================] - 0s 31us/step - loss: 1.1259e-04
Epoch 273/500
2007/2007 [==============================] - 0s 33us/step - loss: 1.1153e-04
Epoch 274/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1225e-04
Epoch 275/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1194e-04
Epoch 276/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1224e-04
Epoch 277/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1382e-04
Epoch 278/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1399e-04
Epoch 279/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1301e-04
Epoch 280/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1266e-04
Epoch 281/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1309e-04
Epoch 282/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1345e-04
Epoch 283/500

2007/2007 [==============================] - 0s 31us/step - loss: 1.1391e-04
Epoch 363/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1081e-04
Epoch 364/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1149e-04
Epoch 365/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1222e-04
Epoch 366/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1238e-04
Epoch 367/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1424e-04
Epoch 368/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1346e-04
Epoch 369/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1205e-04
Epoch 370/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1284e-04
Epoch 371/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1405e-04
Epoch 372/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1289e-04
Epoch 373/500

2007/2007 [==============================] - 0s 31us/step - loss: 1.1427e-04
Epoch 453/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1017e-04
Epoch 454/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1195e-04
Epoch 455/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1180e-04
Epoch 456/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1265e-04
Epoch 457/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1324e-04
Epoch 458/500
2007/2007 [==============================] - 0s 32us/step - loss: 1.1264e-04
Epoch 459/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1330e-04
Epoch 460/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1243e-04
Epoch 461/500
2007/2007 [==============================] - 0s 30us/step - loss: 1.1114e-04
Epoch 462/500
2007/2007 [==============================] - 0s 31us/step - loss: 1.1228e-04
Epoch 463/500

In [18]:
train_predict = model.predict(xtr)
test_predict = model.predict(xts)

In [49]:
predicted_stock = model.predict(xts)
predicted_stock = sc.inverse_transform(predicted_stock)

test_temp = pd.DataFrame(test_Open, columns=['Actual'])
pred_temp = pd.DataFrame(predicted_stock, columns=['Predicted'])
pred_total = pd.concat((pred_temp, test_temp), axis=1)

In [50]:
pred_total

,Predicted,Actual
0,10623.528320,11620.70
1,11474.030273,11566.60
2,9728.682617,11605.85
3,8684.292969,11731.95
4,8661.312500,11744.95
5,9192.974609,11694.75
6,9376.423828,11675.85
7,8327.970703,11751.80
8,6786.628906,11598.75
9,7726.766602,11514.85


In [52]:
test.head()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
23-Aug-18,11582.75,11620.70,11620.70,11546.70,256.16M,0.10%
24-Aug-18,11557.10,11566.60,11604.60,11532.00,225.60M,-0.22%
27-Aug-18,11691.95,11605.85,11700.95,11595.60,205.69M,1.17%
28-Aug-18,11738.50,11731.95,11760.20,11710.50,249.42M,0.40%
29-Aug-18,11691.90,11744.95,11753.20,11678.85,259.35M,-0.40%


In [27]:
# Evaluate 
training_error = model.evaluate(xtr, ytr, verbose=0)
print('{:.10f}'.format(training_error))

0.0000942184


In [28]:
# Evaluate 
test_error = model.evaluate(xts, yts, verbose=0)
print('{:.10f}'.format(test_error))

0.0002643328


In [29]:
test_predict

array([[0.43268692],
       [0.43908378],
       [0.449753  ],
       [0.46083763],
       [0.45772013],
       [0.4583328 ],
       [0.45911407],
       [0.46281877],
       [0.46521917],
       [0.4723701 ],
       [0.47273284],
       [0.4729778 ],
       [0.47456172],
       [0.47703385],
       [0.47155756],
       [0.4556835 ],
       [0.45196134],
       [0.43335307],
       [0.4280497 ],
       [0.42412695],
       [0.42187804],
       [0.42248183],
       [0.4246577 ],
       [0.4189454 ],
       [0.40374944],
       [0.4040538 ],
       [0.40541932],
       [0.39470974],
       [0.40011916],
       [0.40819025],
       [0.4067155 ],
       [0.40533736],
       [0.4212809 ],
       [0.424667  ],
       [0.420762  ],
       [0.39731658],
       [0.40343022],
       [0.41034403],
       [0.41159785],
       [0.38344318],
       [0.37634072],
       [0.36731339],
       [0.34192783],
       [0.36377645],
       [0.36735198],
       [0.35568985],
       [0.36913922],
       [0.371

In [30]:
train.head()

,Price,Open,High,Low
0,5441.95,5399.15,5447.15,5372.20
1,5449.10,5441.90,5477.50,5435.15
2,5418.60,5446.55,5466.25,5409.20
3,5430.60,5420.20,5450.95,5407.20
4,5397.55,5436.10,5447.85,5386.25


In [31]:
train.tail()

,Price,Open,High,Low
2003,11435.10,11381.70,11452.45,11370.80
2004,11385.05,11397.15,11449.85,11366.25
2005,11470.75,11437.15,11486.45,11431.80
2006,11551.75,11502.10,11565.30,11499.65
2007,11570.90,11581.40,11581.40,11539.85
